# Tutorial on using suprême healsparse survey property maps for DC2 DR6

This tutorial explains how to use the HealSparse survey coverage maps generated by the `suprême` code.  For more information on HealSparse see https://healsparse.readthedocs.io/en/latest/ and for `suprême` see https://github.com/LSSTDESC/supreme/tree/master/supreme.

In brief, a HealSparse map is a sparse implementation of HealPix that allows large maps to be stored efficiently on disk and in memory, without the overhead of `healpy` which requires full sky maps to be stored.  Therefore, we can generate survey property maps at nside=32768 (6" pixels) for DC2 DR6 easily without blowing up your memory.  HealSparse also has easy tools to degrade maps to lower resolutions which can aid visualization.

In [ ]:
import numpy as np
import healsparse
import matplotlib.pyplot as plt
import healpy as hp
import glob
import os

In [ ]:
# Look at the maps that are available
files = sorted(glob.glob('/global/cscratch1/sd/erykoff/dc2_dr6/supreme/*v2*.hs'))
for file in files:
    print(os.path.basename(file))

Looking at the i-band, for example, we have the following maps:
- airmass_wmean (weighted-mean airmass)
- bgmean_wmean (weighted-mean calexp background)
- boresight_dist_wmean (weighted-mean distance from boresight center)
- coadd_mask_or (logical or of all values in the coadd mask plane in this pixel)
- dcr_e1_wmean (weighted-mean of expected contribution of differential chromatic refraction to PSF ellipticity e1)
- dcr_e2_wmean (weighted-mean of expected contribution of differential chromatic refraction to PSF ellipticity e2)
- exptime_sum (sum of exposure time)
- nexp_sum (sum of number of exposures)
- psf_e1_wmean (weighted-mean PSF model ellipticity e1)
- psf_e2_wmean (weighted-mean PSF model ellipticity e2)
- psf_size_wmean (weighted-mean PSF model size)

Let us start by looking at the i-band exposure time in the DC2 DR6 footprint.

In [ ]:
# Load the map
exptime_i = healsparse.HealSparseMap.read('/global/cscratch1/sd/erykoff/dc2_dr6/supreme/supreme_dc2_dr6d_v2_i_exptime_sum.hs')

In [ ]:
# This can be used as-is, but initial plotting will be easier at lower resolution
exptime_i_dg = exptime_i.degrade(nside_out=8192)

In [ ]:
# HealSparse maps have a concept of "valid pixels" that have measurements.  We want to know the valid pixels
# as well as the positions of these in the map
valid_pixels, ra, dec = exptime_i_dg.valid_pixels_pos(lonlat=True, return_pixels=True)

In [ ]:
# And we can plot this.  You can look up values in a HealSparse map by their pixel number.  Invalid values will give hp.UNSEEN
plt.hexbin(ra, dec, C=exptime_i_dg[valid_pixels])
plt.xlim(75,48)
plt.title('Exposure Time (i-band)')
plt.xlabel('RA')
plt.ylabel('Dec')
plt.colorbar()

In [ ]:
# If we zoom in on a region with the degraded map...
plt.hexbin(ra, dec, C=exptime_i_dg[valid_pixels], extent=[64.5, 65, -36, -35.5], gridsize=50)
plt.xlim(65, 64.5)
plt.title('Exposure Time (i-band) (zoom, 8192)')
plt.xlabel('RA')
plt.ylabel('Dec')
plt.colorbar()

In [ ]:
# But we can also do this at full resolution
# Note that there is still some degradation because we're using hexbin which bins things down.
valid_pixels_full, ra_full, dec_full = exptime_i.valid_pixels_pos(lonlat=True, return_pixels=True)
plt.hexbin(ra_full, dec_full, C=exptime_i[valid_pixels_full], extent=[64.5, 65, -36, -35.5], gridsize=200)
plt.xlim(65, 64.5)
plt.title('Exposure Time (i-band) (zoom, 32768)')
plt.xlabel('RA')
plt.ylabel('Dec')
plt.colorbar()

## Using the maps for lookup

Let's say you have a set of ra/dec positions and you want to know the exptime.  Or you want to compute the correlation of some galaxy property with the exposure time, etc.  This is easy!

In [ ]:
rra = np.random.uniform(size=100000, low=64.0, high=65.0)
rdec = np.random.uniform(size=100000, low=-36.0, high=-35.0)

In [ ]:
exptime_pts = exptime_i.get_values_pos(rra, rdec, lonlat=True)

In [ ]:
plt.hist(exptime_pts, bins=30)
plt.xlabel('Exposure Time (s)')

In [ ]:
# And we can do this with another map for the same points
e1_i = healsparse.HealSparseMap.read('/global/cscratch1/sd/erykoff/dc2_dr6/supreme/supreme_dc2_dr6d_v2_i_psf_e1_wmean.hs')

In [ ]:
plt.hexbin(exptime_i.get_values_pos(rra, rdec, lonlat=True),
           e1_i.get_values_pos(rra, rdec, lonlat=True), bins='log', gridsize=30)
plt.xlabel('Exposure Time (i-band) (sec)')
plt.ylabel('PSF Model e1 (i-band) (sec)')